In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Activation, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.applications import ResNet50V2, ResNet101V2, DenseNet121, VGG16, VGG19

2022-05-08 23:28:25.423593: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64/usr/local/cuda-10.2/lib64:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64
2022-05-08 23:28:25.423675: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64/usr/local/cuda-10.2/lib64:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64
2022-05-08 23:28:25.423684: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
def load_CIFAR10(num_class):
    """
    Load CIFAR10 dataset and data preprocessing.
    :param data_dir: Path where to find the data file.
    :return: Tuple of numpy arrays: (x_train, y_train), (x_test, y_test).
    """
    # Initialize Data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

    x_train = x_train / 255.0
    x_test = x_test / 255.0
    
    y_train = tf.keras.utils.to_categorical(y_train, num_class)
    y_test = tf.keras.utils.to_categorical(y_test, num_class)
    
    return (x_train, y_train), (x_test, y_test)

(x, y), (x_test, y_test) = load_CIFAR10(100)

def clipDataTopX(dataToClip, top=3):
	res = [ sorted(s, reverse=True)[0:top] for s in dataToClip ]
	return np.array(res)

In [3]:
x[0].shape

(32, 32, 3)

In [6]:
def get_Resnet50(input_shape=x[0].shape, num_classes=100):
    """Define and return the ResNet50V2 model.
    Args:
        input_shape: a tensor defined the data shape.
        num_classes: the number of classes.
    Returns: 
        a keras model object named ResNet50V2.
    """
    model = tf.keras.Sequential([
        ResNet50V2(include_top=False,
                   weights='imagenet',
                   input_shape=input_shape),
        GlobalAveragePooling2D(),
        Dense(num_classes),
        Activation('softmax')
    ])
    model.summary()
    
    return model

model = get_Resnet50(x[0].shape, num_classes=100)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 1, 1, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               204900    
_________________________________________________________________
activation_3 (Activation)    (None, 100)               0         
Total params: 23,769,700
Trainable params: 23,724,260
Non-trainable params: 45,440
_________________________________________________________________


In [9]:
initial_model = get_Resnet50()
for j in range(2):
    deltas = []
    for i in range(4):
        model = get_Resnet50()
        model.set_weights(initial_model.get_weights())
        model.compile(loss='categorical_crossentropy',
                      optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6),
                      metrics=['accuracy'])
        model.fit(x[i*10000: i*10000+10000],
                  y[i*10000: i*10000+10000],
                  batch_size=64,
                  epochs=10,
                  verbose=1)
        delta = np.array(initial_model.get_weights()) - \
            np.array(model.get_weights())
        deltas.append(delta)
        if j == 0:
            model.save('./model/keras_cifar100_trained_model_' + str(i) + '.h5')
    delt_av = (deltas[0] + deltas[1] + deltas[2] + deltas[3]) / 4
    new_weights = np.array(initial_model.get_weights()) - delt_av
    initial_model.set_weights(new_weights)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 1, 1, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_9 ( (None, 2048)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               204900    
_________________________________________________________________
activation_9 (Activation)    (None, 100)               0         
Total params: 23,769,700
Trainable params: 23,724,260
Non-trainable params: 45,440
_________________________________________________________________
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 1, 1, 2048)        23564800  
______________

/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 1, 1, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_11  (None, 2048)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 100)               204900    
_________________________________________________________________
activation_11 (Activation)   (None, 100)               0         
Total params: 23,769,700
Trainable params: 23,724,260
Non-trainable params: 45,440
_________________________________________________________________
Train on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 16s 2ms/sample - loss: 4.6436 - accuracy: 0.0237
Epoch 2/10
10000/10000 [==============================] - 7s 742us/sample - loss: 4.3070 - accuracy: 0.0

/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 1, 1, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_12  (None, 2048)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 100)               204900    
_________________________________________________________________
activation_12 (Activation)   (None, 100)               0         
Total params: 23,769,700
Trainable params: 23,724,260
Non-trainable params: 45,440
_________________________________________________________________
Train on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 15s 2ms/sample - loss: 4.6304 - accuracy: 0.0266
Epoch 2/10
10000/10000 [==============================] - 8s 762us/sample - loss: 4.2317 - accuracy: 0.0

...

/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 1, 1, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_16  (None, 2048)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 100)               204900    
_________________________________________________________________
activation_16 (Activation)   (None, 100)               0         
Total params: 23,769,700
Trainable params: 23,724,260
Non-trainable params: 45,440
_________________________________________________________________
Train on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 15s 2ms/sample - loss: 2.6625 - accuracy: 0.3344
Epoch 2/10
10000/10000 [==============================] - 7s 720us/sample - loss: 1.8922 - accuracy: 0.5

/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 1, 1, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_17  (None, 2048)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 100)               204900    
_________________________________________________________________
activation_17 (Activation)   (None, 100)               0         
Total params: 23,769,700
Trainable params: 23,724,260
Non-trainable params: 45,440
_________________________________________________________________
Train on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 18s 2ms/sample - loss: 2.6312 - accuracy: 0.3387
Epoch 2/10
10000/10000 [==============================] - 8s 758us/sample - loss: 1.8698 - accuracy: 0.5

/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or 

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

x_train = np.vstack((x[30000:40000], x_test))
x_train = clipDataTopX(initial_model.predict(x_train), 3)
y_train = np.zeros(30000)
for i in range(10000):
    y_train[i] = 1

attack = RandomForestClassifier(n_estimators=50)
attack.fit(x_train, y_train[0:20000])
xt = initial_model.predict(np.vstack((x[0:20000], x_test)))
xt = clipDataTopX(xt, 3)
y_pred = attack.predict(xt)
print(classification_report(y_true=y_train, y_pred=y_pred))
print(roc_auc_score(y_train, y_pred))

              precision    recall  f1-score   support

         0.0       0.75      0.75      0.75     20000
         1.0       0.50      0.48      0.49     10000

    accuracy                           0.66     30000
   macro avg       0.62      0.62      0.62     30000
weighted avg       0.66      0.66      0.66     30000

0.619825


In [ ]:
x_train.shape

(20000, 3)